In [1]:
PATH_NEWS_ARTICLES="News.csv"
ARTICLES_READ=[3,7,8,17,18]
NUM_RECOMMENDED_ARTICLES=500

In [2]:
# installing libraries
try:
    
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    
    import nltk
    import hazm
    normalizer = hazm.Normalizer()
except ImportError:
    print('You are missing some packages! ' \
          'We will try installing them before continuing!')
    !pip install --user "numpy" "pandas" "sklearn" "nltk"
    import numpy
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    import re
    
    
    
    print('Done!')

In [3]:
news_articles = pd.read_csv(PATH_NEWS_ARTICLES,encoding="utf8")
news_articles.head()

,number,link,title,news
0,1,https://www.farsnews.ir//news/13991115001206/ت...,توجه به ظرفیت معادن طلا و مس سبب تکمیل زیرساخت...,\r\n\r\r\n ...
1,2,https://www.farsnews.ir//news/13991115001210/ع...,عدم ارتباط مشخص میان«علم‌محوری»، «انسان‌محوری»...,\r\n\r\r\n ...
2,3,https://www.farsnews.ir//news/13991115001178/ا...,افتتاح طرح‌های پرورش شترمرغ و بسته‌بندی عسل در...,\r\n\r\r\n ...
3,4,https://www.farsnews.ir//news/13991115001207/آ...,آزادی 42 زندانی نیازمند در مشهد,\r\n\r\r\n ...
4,5,https://www.farsnews.ir//news/13991115001199/ا...,اختصاص ۳ میلیارد ریال اعتبار به ۳ شهرداری منطق...,\r\n\r\r\n ...


In [4]:
#Select relevant columns and remove rows with missing values
news_articles = news_articles[['number','title','news']].dropna()
#articles is a list of all articles
articles = news_articles['news'].tolist()
articles[0] #an uncleaned article

'\r\n\r\r\n                                        استاندار خراسان رضوی گفت: توجه به ظرفیت معادن طلا و مس، راهکاری مطلوب در راستای تکمیل زیرساخت منطقه میان جلگه نیشابور است.\r\r\n                                    به گزارش خبرنگار فارس از نیشابور، محمدصادق معتمدیان امشب در جلسه\u200cای که در امامزاده عشق آباد برگزار شد، اظهار کرد: توجه ویژه ما به مناطق کمتر توسعه یافته و مناطق تازه تأسیس در بخش تکمیل زیرساخت\u200cها است.استاندار خراسان رضوی ادامه داد: با شناسایی نارسایی\u200cها و مطالبات مردمی می\u200cتوانیم به وظایف خودمان را با وجود شرایط تحریم و مشکلات جدی مالی با توجه به ظرفیت\u200cها و پتانسیل\u200cهای این استان پهناور و ظرفیت معادن، کشاورزی و اقتصاد زیارتی به حل مشکلات منطقه بپردازیم.وی با بیان اینکه بخش میان جلگه در حوزه معادن از ظرفیت خوبی برخوردار است، افزود: استانداری باید در حوزه معادن این منطقه مطالعات بیشتری انجام داده و توجه ویژه\u200cای به اکتشاف معادن میان جلگه داشته باشد، زیرا میان جلگه با برخورداری از معادن طلا و مس ظرفیت بالایی برای رسیدگی به مطالبات مردمی دارد.معتم

In [5]:
from hazm import *
from __future__ import unicode_literals
# https://www.sobhe.ir/hazm/

In [6]:
# tokenizing news article
def clean_tokenize(document):
    document=normalizer.normalize(document)
    tokens = hazm.word_tokenize(document)              #Tokenize sentences
    cleaned_article = ' '.join([item for item in tokens])    #Stemming each token
    return cleaned_article

In [7]:
cleaned_articles = list (map(clean_tokenize, articles[0:500]))
cleaned_articles  #a cleaned, tokenized  

['استاندار خراسان رضوی گفت : توجه به ظرفیت معادن طلا و مس ، راهکاری مطلوب در راستای تکمیل زیرساخت منطقه میان جلگه نیشابور است . به گزارش خبرنگار فارس از نیشابور ، محمدصادق معتمدیان امشب در جلسه\u200cای که در امامزاده عشق آباد برگزار شد ، اظهار کرد : توجه ویژه ما به مناطق کمتر توسعه یافته و مناطق تازه تأسیس در بخش تکمیل زیرساخت\u200cها است . استاندار خراسان رضوی ادامه داد : با شناسایی نارسایی\u200cها و مطالبات مردمی می\u200cتوانیم به وظایف خودمان را با وجود شرایط تحریم و مشکلات جدی مالی با توجه به ظرفیت\u200cها و پتانسیل\u200cهای این استان پهناور و ظرفیت معادن ، کشاورزی و اقتصاد زیارتی به حل مشکلات منطقه بپردازیم . وی با بیان اینکه بخش میان جلگه در حوزه معادن از ظرفیت خوبی برخوردار است ، افزود : استانداری باید در حوزه معادن این منطقه مطالعات بیشتری انجام داده و توجه ویژه\u200cای به اکتشاف معادن میان جلگه داشته_باشد ، زیرا میان جلگه با برخورداری از معادن طلا و مس ظرفیت بالایی برای رسیدگی به مطالبات مردمی دارد . معتمدیان در خصوص مشکلات مالی شهرداری عشق\u200cآباد بیان کرد : در یک بازه زمان

In [8]:
#Get user representation in terms of words associated with read articles
user_articles = ' '.join(cleaned_articles[i] for i in ARTICLES_READ)

In [9]:
user_articles

'در سالروز ولادت حضرت زهرا ( س ) و مقارن با ایام دهه مبارک فجر ۴۲ زندانی نیازمند با همت خیرین از زندان مرکزی مشهد آزاد شدند . به گزارش خبرگزاری فارس از مشهد به نقل از روابط عمومی اداره کل زندان\u200cهای خراسان رضوی ، در سالروز ولادت حضرت زهرا ( س ) و مقارن با ایام دهه مبارک فجر ۴۲ زندانی نیازمند با همت خیرین از زندان مرکزی مشهد آزاد شدند . مدیرکل زندان\u200cهای خراسان رضوی در آئین آزادی این تعداد از زندانیان گفت : در راستای تکالیف سازمانی و اجتماعی همزمان با چهل و دومین سالگرد پیروزی انقلاب اسلامی با مشارکت پرمهر خیرین زمینه آزادی ۴۲ زندانی نیازمند شامل ۴ زن و ۳۸ مرد از زندان مرکزی مشهد فراهم شد . جانفدا افزود : مبلغ ۵ میلیارد ریال بدهی این تعداد از زندانیان با همت مبلغین حوزه علمیه خراسان که در طرح معراج زندان فعال هستند و همچنین فاطمیه آذربایجانی\u200cهای مقیم مشهد تأمین و پرداخت شد . وی ضمن تقدیر از توجه خیرین و آحاد مردم به موضوع زندان و زندانی ، گفت : همه ارکان اجتماع در قبال زندانیان که آسیب\u200cدیده همین جامعه هستند مسئولند و باید این مسئله به یک گفتمان مثبت اجتماعی تبدیل شود .

In [10]:

path="/Users/nojab/Documents/pythonproject/persian"
persian= open(path,encoding="utf8")
stop_words = persian


In [11]:

persian_stop_word =list()
for line in persian:
    line = line.strip('\n')
    line = line.strip('\t')
    persian_stop_word.append(line)
persian_stop_word

['\ufeff!',
 '"',
 '#',
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 ':',
 '[',
 ']',
 '«',
 '»',
 '،',
 '؛',
 '؟',
 'آباد',
 'آخ',
 'آخر',
 'آخرها',
 'آخه',
 'آدمهاست',
 'آرام',
 'آرام آرام',
 'آره',
 'آری',
 'آزادانه',
 'آسان',
 'آسیب پذیرند',
 'آشنایند',
 'آشکارا',
 'آقا',
 'آقای',
 'آقایان',
 'آمد',
 'آمدن',
 'آمده',
 'آمرانه',
 'آن',
 'آن گاه',
 'آنان',
 'آنانی',
 'آنجا',
 'آنرا',
 'آنطور',
 'آنقدر',
 'آنها',
 'آنهاست',
 'آنچنان',
 'آنچنان که',
 'آنچه',
 'آنکه',
 'آنگاه',
 'آن\u200cها',
 'آهان',
 'آهای',
 'آور',
 'آورد',
 'آوردن',
 'آورده',
 'آوه',
 'آی',
 'آیا',
 'آید',
 'آیند',
 'ا',
 'اتفاقا',
 'اثرِ',
 'اجراست',
 'احتراما',
 'احتمالا',
 'احیاناً',
 'اخیر',
 'اخیراً',
 'اری',
 'از',
 'از آن پس',
 'از جمله',
 'ازاین رو',
 'ازجمله',
 'ازش',
 'اساسا',
 'اساساً',
 'است',
 'استفاد',
 'استفاده',
 'اسلامی اند',
 'اش',
 'اشتباها',
 'اشکارا',
 'اصلا',
 'اصلاً',
 'اصولا',
 'اصولاً',
 'اعلام',
 'اغلب',
 'افزود',
 'افسوس',
 'اقل',
 'اقلیت',
 'الا',
 'الان',
 'البته',
 'البتّه',
 'الهی',
 'ال

In [12]:
#Generate tfidf matrix model for entire corpus
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_matrix = TfidfVectorizer(sublinear_tf=True,max_df=0.5,stop_words=persian_stop_word, min_df=2)
article_tfidf_matrix = tfidf_matrix.fit_transform(cleaned_articles)
article_tfidf_matrix #tfidf vector of an article

C:\Users\nojab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آسانی', 'آسیب', 'اثر', 'احیانا', 'اخیرا', 'ازاین', 'اسلامی', 'اطلاعند', 'الاسف', 'البت', 'الظاهر', 'بااین', 'بازی', 'باوجودی', 'برآن', 'برنامه', 'بسته', 'بطوری', 'بفهمی', 'تازگی', 'تحریم', 'تدریج', 'ترتیب', 'تردید', 'ترند', 'تصریحا', 'تفاوتند', 'تند', 'توؤما', 'تول', 'تک', 'تی', 'ثالثا', 'ثانی', 'جمع', 'جمله', 'جنس', 'جوری', 'حاشیه', 'حسابی', 'حکما', 'خارج', 'خسته', 'خوبی', 'خودبه', 'خودی', 'داند', 'دراین', 'درحالی', 'درشتی', 'درصورتی', 'درعین', 'درپی', 'دسته', 'دلخواه', 'دنبال', 'دیوانه', 'ذاتا', 'ذلک', 'راجع', 'رسد', 'رغم', 'رود', 'روزه', 'روشنی', 'زده', 'زودی', 'ساده', 'سادگی', 'سازهاست', 'سالم', 'ساله', 'ست', 'سرعت', 'سمت', 'سه', 'سهوا', 'سیاه', 'شاأالله', 'شدت', 'صریحا', 'صندوق', 'ضد', 'ضمن

<500x5947 sparse matrix of type '<class 'numpy.float64'>'
	with 56336 stored elements in Compressed Sparse Row format>

In [13]:
#Generate tfidf matrix model for read articles
user_article_tfidf_vector = tfidf_matrix.transform([user_articles])
user_article_tfidf_vector

<1x5947 sparse matrix of type '<class 'numpy.float64'>'
	with 344 stored elements in Compressed Sparse Row format>

In [14]:
user_article_tfidf_vector.toarray()

array([[0.        , 0.        , 0.03489713, ..., 0.        , 0.        ,
        0.        ]])

In [15]:
# Calculate cosine similarity between user read articles and unread articles

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
articles_similarity_score=cosine_similarity(article_tfidf_matrix, user_article_tfidf_vector)

In [17]:
recommended_articles_id = articles_similarity_score.flatten().argsort()[:5]

In [18]:
recommended_articles_id

array([389, 245, 234, 377, 330], dtype=int64)

In [19]:
#Remove read articles from recommendations
final_recommended_articles_id = [article_id for article_id in recommended_articles_id 
                                 if article_id not in ARTICLES_READ ][:NUM_RECOMMENDED_ARTICLES]

In [20]:
# Get the recommended news article
final_recommended_articles_id

[389, 245, 234, 377, 330]

In [21]:
#Recommended Articles and their title
print ('Articles Read')
print (news_articles.loc[news_articles['number'].isin(ARTICLES_READ)]['title'])
print ('\n')
print ('Recommender ')
print (news_articles.loc[news_articles['number'].isin(final_recommended_articles_id)]['title'])

Articles Read
2     افتتاح طرح‌های پرورش شترمرغ و بسته‌بندی عسل در...
6     توضیحات دادسرای مشهد درباره ماجرای پارک ملت و ...
7          تجلیل از مادران گیلانی دارای ۳ فرزند و بیشتر
16    طرح دولت ترامپ برای کاهش نیروهای آمریکا در آلم...
17    لیگ فوتبال بلژیک| برد پرگل خنت در حضور میلاد م...
Name: title, dtype: object


Recommender 
233    سردار نجات: با اعتماد به نیروهای جوان، صنعت ما...
244    محدودیتهای بیشتر برای مقابله با کرونا در کشوره...
329    دادگاه لاهه مسئول رسیدگی به پرونده اراضی فلسطی...
376                 بهترین بازیکن ماه سری‌آ مشخص شد +عکس
388    سیف‌الله‌پور: اطرافیان فکری اخبار غلط به او می...
Name: title, dtype: object
